In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sqlalchemy import create_engine, inspect
from dotenv import load_dotenv
import os
import logging
import pickle

In [3]:
logging.basicConfig(level=logging.INFO)

def load_all_dataframes():
    try:
        grammy_df = pd.read_pickle('./grammy_ready_df.pkl')
        spotify_df = pd.read_pickle('./spotify_ready_df.pkl')
        
        logging.info(f"Grammy dataframe: \n{grammy_df.head(5)}")
        logging.info(f"Spotify dataframe: \n{spotify_df.head(5)}")
    
        
        logging.info("Dataframes loaded successfully")
        
        return grammy_df, spotify_df
        
        
    except Exception as e:
        logging.error(f"Error loading dataframes: {e}")

grammy, spotify = load_all_dataframes()

INFO:root:Grammy dataframe: 
   year                              title              published_at  \
0  2019  62nd Annual GRAMMY Awards  (2019) 2020-05-19 12:10:28+00:00   
1  2019  62nd Annual GRAMMY Awards  (2019) 2020-05-19 12:10:28+00:00   
2  2019  62nd Annual GRAMMY Awards  (2019) 2020-05-19 12:10:28+00:00   
3  2019  62nd Annual GRAMMY Awards  (2019) 2020-05-19 12:10:28+00:00   
4  2019  62nd Annual GRAMMY Awards  (2019) 2020-05-19 12:10:28+00:00   

                 updated_at            category     nominee         artist  \
0 2020-05-19 12:10:28+00:00  Record Of The Year     Bad Guy  Billie Eilish   
1 2020-05-19 12:10:28+00:00  Record Of The Year     Hey, Ma       Bon Iver   
2 2020-05-19 12:10:28+00:00  Record Of The Year     7 rings  Ariana Grande   
3 2020-05-19 12:10:28+00:00  Record Of The Year  Hard Place         H.E.R.   
4 2020-05-19 12:10:28+00:00  Record Of The Year        Talk         Khalid   

                                             workers  winner  
0  Fin

In [4]:
def merge(grammy_df, spotify_df):
    try:
        logging.info("Merging dataframes...")
        
        merged_df = pd.merge(grammy_df, spotify_df, left_on='artist', right_on='artists', how='inner')

        merged_df['artist'] = merged_df['artist'].combine_first(merged_df['artists'])
        
        if 'winner' in merged_df.columns:
            valores_unicos = merged_df['winner'].unique()
            logging.info(f"Unique values in 'winner' column: {valores_unicos}")

            conteo_registros = merged_df['winner'].value_counts()
            for valor, conteo in conteo_registros.items():
                logging.info(f"Total records with value '{valor}': {conteo}")

            merged_df = merged_df.drop(columns=['winner'])
            logging.info("Column 'winner' removed from the dataset.")

        merged_df.drop(columns=['artists','liveness','time_signature', 'loudness','mode'], inplace=True)

        logging.info("Merge completed successfully.")
        return merged_df

    except Exception as e:
        logging.error(f"Error during dataframe merge: {e}")


df_merge = merge(grammy, spotify)

INFO:root:Merging dataframes...
INFO:root:Unique values in 'winner' column: [ True]
INFO:root:Total records with value 'True': 23851
INFO:root:Column 'winner' removed from the dataset.
INFO:root:Merge completed successfully.


In [5]:
def save_csv(mergedf):
    try:
        # num_duplicates = mergedf.duplicated().sum()
        # logging.info(f"The total of duplicates in the dataframe is: {num_duplicates}")
        mergedf.to_csv('../Clean-Data/merge_df.csv', index=False)
        logging.info("Dataframe saved as CSV successfully.")
        
    except Exception as e:
        logging.error(f"Error saving dataframe as CSV: {e}")
        
save_csv(df_merge)

In [6]:
def save_DB(df):
    try:
        load_dotenv()

        localhost = os.getenv('LOCALHOST')
        port = os.getenv('PORT')
        nameDB = os.getenv('DB_NAME')
        userDB = os.getenv('DB_USER')
        passDB = os.getenv('DB_PASS')

        engine = create_engine(f'postgresql+psycopg2://{userDB}:{passDB}@{localhost}:{port}/{nameDB}')
        inspector = inspect(engine)

        with engine.connect() as connection:
            logging.info("Successfully connected to the database.")
            
            try:
                df.to_sql('data_merge', engine, if_exists='replace', index=False)
                logging.info("Table 'data_merge' added.")

            except Exception as e:
                logging.error(f"Error adding data: {e}")

    except Exception as e:
        logging.error(f"Error connecting to the database: {e}")
        
    finally:
        engine.dispose()
        logging.info("Database connection closed.")
        
save_DB(df_merge)

In [8]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
import logging

def authenticate():
    gauth = GoogleAuth()

    gauth.LoadClientConfigFile("../client_secret.json")
    
    gauth.LocalWebserverAuth()
    
    return gauth

def save_drive(df, file_name='df_merge.csv', folder_id=None):
    try:
        temp_file_path = f"./{file_name}"
        df.to_csv(temp_file_path, index=False)
        logging.info("DataFrame saved as CSV successfully.")

        gauth = authenticate()
        drive = GoogleDrive(gauth)

        file = drive.CreateFile({'title': file_name, 'parents': [{'id': folder_id}] if folder_id else []})
        file.SetContentFile(temp_file_path)
        file.Upload()

        print(f"Archivo '{file_name}' subido correctamente a Google Drive.")

        os.remove(temp_file_path)
        
    except Exception as e:
        print(f"Error al subir el archivo a Google Drive: {e}")

save_drive(df_merge)

INFO:root:DataFrame saved as CSV successfully.


Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?client_id=241896318925-ktad4foffvmcujon4e29vhr73psf082p.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&access_type=offline&response_type=code



INFO:oauth2client.client:Successfully retrieved access token
INFO:googleapiclient.discovery_cache:file_cache is only supported with oauth2client<4.0.0


Authentication successful.
Archivo 'df_merge.csv' subido correctamente a Google Drive.
